In [2]:
!ls /dev/ttyUSB*

/dev/ttyUSB0


In [4]:
# building the robot
import requests
from rosmaster_create_bot import CreateBot, sleep, EXTENSION_BOARD_ADDRESS, GAMEPAD_CONTROLLER_ID
    # check with:   ls /dev/ttyUSB*
    # if cant find try to reset the device, reinstall and reconnect it
extension_board_address = "/dev/ttyUSB0" or EXTENSION_BOARD_ADDRESS 
bot = CreateBot(extension_board_address)

move_arm = bot.arm
move_motor = bot.motor


libEGL warning: DRI2: failed to authenticate


Rosmaster Serial Opened! Baudrate=115200
arm reset
[DEBUG] Sending angles to robot: [90, 90, 90, 90, 90, 160]
[DEBUG] Sending angles to robot: [90, 84.0, 84.0, 84.0, 90, 160]


In [5]:
#functions

def turn_around(turning_time=1.5):
    move_motor.move_turn0(is_move=True)
    sleep(turning_time)  #timing for turning around
    move_motor.stop()

def move_forward():
    move_motor.move_forward(is_move=True)
    

def stop():
    move_motor.stop()

def lift_arm():
    move_arm.arm_bent_up(70)
def pull_down():
    move_arm.arm_bent_down(70)

def turn_off_the_lamp():
    url = "https://arthurcam.com/api/arduinoIOT/2"
    requests.get(url)


In [6]:
#function running for demo

#preset
pull_down()

turn_around()
sleep()
move_forward()
sleep()
stop()
sleep()
lift_arm()
sleep()
pull_down()
sleep()
turn_around()
sleep()
#turn of the lamp

#resture
lift_arm()

[DEBUG] Sending angles to robot: [90, 42.0, 42.0, 42.0, 90, 160]
[DEBUG] Sending angles to robot: [90, 84.0, 84.0, 84.0, 90, 160]
[DEBUG] Sending angles to robot: [90, 42.0, 42.0, 42.0, 90, 160]
[DEBUG] Sending angles to robot: [90, 84.0, 84.0, 84.0, 90, 160]


In [ ]:
# check the API action on https://www.arthurcam.com
turn_off_the_lamp()

In [1]:
#building the model
from nemo_speech import NemoSpeech
nemo_model = NemoSpeech()
#load the recorder
from recorder import VoiceRecorder
record = VoiceRecorder()
#load phrase to function
from phrase_to_function import PhraseToFunction

# api calling
import requests

#building the robot
from rosmaster_create_bot import CreateBot, sleep, EXTENSION_BOARD_ADDRESS, GAMEPAD_CONTROLLER_ID
    # check with:   ls /dev/ttyUSB*
    # if cant find try to reset the device, reinstall and reconnect it
extension_board_address = "/dev/ttyUSB0" or EXTENSION_BOARD_ADDRESS 
bot = CreateBot(extension_board_address)

move_arm = bot.arm
move_motor = bot.motor

libEGL warning: DRI2: failed to authenticate


🔁 Initializing NemoSpeech models...
✅ CUDA is available
📦 Loading model from cache: model_cache/QuartzNet15x5Base-En.nemo
[NeMo I 2025-06-08 01:22:55 features:305] PADDING: 16


[NeMo W 2025-06-08 01:22:55 nemo_logging:361] /usr/lib/python3/dist-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
      "cipher": algorithms.TripleDES,
    
[NeMo W 2025-06-08 01:22:55 nemo_logging:361] /usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 45.0.0.
      "class": algorithms.Blowfish,
    
[NeMo W 2025-06-08 01:22:55 nemo_logging:361] /usr/lib/python3/dist-packages/paramiko/transport.py:261: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.


[NeMo I 2025-06-08 01:22:56 save_restore_connector:275] Model EncDecCTCModel was successfully restored from /home/arthur/Desktop/Nemo/model_cache/QuartzNet15x5Base-En.nemo.
✅ CUDA is available
📦 Loading model from cache: model_cache/tts_en_tacotron2.nemo


[NeMo E 2025-06-08 01:22:56 base:45] `nemo_text_processing` not installed, see https://github.com/NVIDIA/NeMo-text-processing for more details.
[NeMo E 2025-06-08 01:22:56 base:48] The normalizer will be disabled.
[NeMo W 2025-06-08 01:22:58 en_us_arpabet:66] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2025-06-08 01:22:58 modelPT:180] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /raid/datasets/tts_data/train_manifest.json
      sample_rate: 22050
      sup_data_path: null
      sup_data_types: null
      n_fft: 1024
  

[NeMo I 2025-06-08 01:22:58 features:305] PADDING: 16
[NeMo I 2025-06-08 01:22:58 features:314] STFT using exact pad
[NeMo I 2025-06-08 01:22:59 save_restore_connector:275] Model Tacotron2Model was successfully restored from /home/arthur/Desktop/Nemo/model_cache/tts_en_tacotron2.nemo.
✅ CUDA is available
📦 Loading model from cache: model_cache/tts_en_hifigan.nemo


[NeMo W 2025-06-08 01:23:00 modelPT:180] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2025-06-08 01:23:00 modelPT:187] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2025-06-08 01:23:00 features:305] PADDING: 0


[NeMo W 2025-06-08 01:23:00 features:282] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2025-06-08 01:23:00 features:305] PADDING: 0
[NeMo I 2025-06-08 01:23:02 save_restore_connector:275] Model HifiGanModel was successfully restored from /home/arthur/Desktop/Nemo/model_cache/tts_en_hifigan.nemo.
✅ All models loaded
🎧 Total input devices: 24 ['[4] NVIDIA Jetson Orin NX APE: - (hw:1,0)', '[5] NVIDIA Jetson Orin NX APE: - (hw:1,1)', '[6] NVIDIA Jetson Orin NX APE: - (hw:1,2)', '[7] NVIDIA Jetson Orin NX APE: - (hw:1,3)', '[8] NVIDIA Jetson Orin NX APE: - (hw:1,4)', '[9] NVIDIA Jetson Orin NX APE: - (hw:1,5)', '[10] NVIDIA Jetson Orin NX APE: - (hw:1,6)', '[11] NVIDIA Jetson Orin NX APE: - (hw:1,7)', '[12] NVIDIA Jetson Orin NX APE: - (hw:1,8)', '[13] NVIDIA Jetson Orin NX APE: - (hw:1,9)', '[14] NVIDIA Jetson Orin NX APE: - (hw:1,10)', '[15] NVIDIA Jetson Orin NX APE: - (hw:1,11)', '[16] NVIDIA Jetson Orin NX APE: - (hw:1,12)', '[17] NVIDIA Jetson Orin NX APE: - (hw:1,13)', '[18] NVIDIA Jetson Orin NX APE: - (hw:1,14)', '[19] NVIDIA Jetson Orin NX APE: - (hw:1,15)',

In [ ]:
# appling Rosmater commands for the demo
# agent waiting loop for command with exiting
from agent import Agent
from recorder import VoiceRecorder
from phrase_to_function import PhraseToFunction
import time

#speach command
def say(text: str):
    nemo_model.text_to_speech(text, play=True)

# Microphone, Input callback: returns transcribed text
recorder = VoiceRecorder()

def asr_input(): #resouces callback
    recorder.record()
    audio = recorder.get_audio()
    if len(audio) == 0:
        return ""
    return nemo_model.transcribe_from_audio(audio).text.strip().lower()


#functions
def turn_around(turning_time=1.5):
    move_motor.move_turn0(is_move=True)
    sleep(turning_time)  #timing for turning around
    move_motor.stop()

def move_forward():
    say("Yes moving forward")
    move_motor.move_forward(is_move=True)
    
def stop():
    move_motor.stop()

def lift_arm():
    say("Arm up")
    move_arm.arm_bent_up(70)


def pull_down():
    say("Arm down")
    move_arm.arm_bent_down(70)

def turn_off_the_lamp():
    url = "https://arthurcam.com/api/arduinoIOT/2"
    requests.get(url)


EXIT_COMMANDS = {"exit", "quit", "terminate"}

rosmaster_commands = {
    "say hi": lambda: say("Hi Rosmaster"),
    "turn around": turn_around,
    "move forward": move_forward,
    "stop": stop,
    "lift arm": lift_arm,
    "pull down": pull_down,
    "arm down": pull_down,
    "turn off the lamp": turn_off_the_lamp,
    "make beep": lambda: bot.beep()
}


# Logic callback for the agent
def agent_logic(commands, input_callback):
    recognizer = PhraseToFunction(commands)
    say("Rosmaster is waiting for the command")
    while True:
        try:
            command = input_callback()
            if not command:
                bot.beep()
                continue
            print(f"🗣 Command: '{command}'")

            if command in EXIT_COMMANDS:
                say("Exiting. Goodbye.")
                bot.beep()
                break

            recognized, result = recognizer.execute(command)
            if not recognized:
                bot.beep()
            if result:
                pass
                
        except Exception as e:
            bot.beep()
            print(f"❌ Error: {e}")
            say("Error occurred. Try again.")
            bot.beep()
            time.sleep(1)


if __name__ == "__main__":
    stop()
    pull_down()#preset command
    main_agent = Agent("MainAgent", agent_logic, asr_input, rosmaster_commands)
    main_agent.run()
    lift_arm()#reset command
    bot.terminate()


[NeMo W 2025-06-08 01:23:02 tacotron2:144] parse() is meant to be called in eval mode.
[NeMo W 2025-06-08 01:23:02 tts_tokenizers:663] Text: [​ AA1RM DAW1N​ ] contains unknown char/phoneme: [​].Original text: [​ Arm down​ ]. Symbol will be skipped.
[NeMo W 2025-06-08 01:23:02 tts_tokenizers:663] Text: [​ AA1RM DAW1N​ ] contains unknown char/phoneme: [​].Original text: [​ Arm down​ ]. Symbol will be skipped.


🎧 Total input devices: 24 ['[4] NVIDIA Jetson Orin NX APE: - (hw:1,0)', '[5] NVIDIA Jetson Orin NX APE: - (hw:1,1)', '[6] NVIDIA Jetson Orin NX APE: - (hw:1,2)', '[7] NVIDIA Jetson Orin NX APE: - (hw:1,3)', '[8] NVIDIA Jetson Orin NX APE: - (hw:1,4)', '[9] NVIDIA Jetson Orin NX APE: - (hw:1,5)', '[10] NVIDIA Jetson Orin NX APE: - (hw:1,6)', '[11] NVIDIA Jetson Orin NX APE: - (hw:1,7)', '[12] NVIDIA Jetson Orin NX APE: - (hw:1,8)', '[13] NVIDIA Jetson Orin NX APE: - (hw:1,9)', '[14] NVIDIA Jetson Orin NX APE: - (hw:1,10)', '[15] NVIDIA Jetson Orin NX APE: - (hw:1,11)', '[16] NVIDIA Jetson Orin NX APE: - (hw:1,12)', '[17] NVIDIA Jetson Orin NX APE: - (hw:1,13)', '[18] NVIDIA Jetson Orin NX APE: - (hw:1,14)', '[19] NVIDIA Jetson Orin NX APE: - (hw:1,15)', '[20] NVIDIA Jetson Orin NX APE: - (hw:1,16)', '[21] NVIDIA Jetson Orin NX APE: - (hw:1,17)', '[22] NVIDIA Jetson Orin NX APE: - (hw:1,18)', '[23] NVIDIA Jetson Orin NX APE: - (hw:1,19)', '[24] Jabra EVOLVE 20 MS: USB Audio (hw:2,0)', '[

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
[NeMo W 2025-06-08 01:23:04 tts_tokenizers:663] Text: [​ rosmaster is WEY1TIH0NG for the KAH0MAE1ND​ ] contains unknown char/phoneme: [​].Original text: [​ Rosmaster is waiting for the command​ ]. Symbol will be skip

[DEBUG] Sending angles to robot: [90, 42.0, 42.0, 42.0, 90, 160]


ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred


...

🗣 Command: 'toron of de lamb'
[Adapt] ❌ No confident match for: 'toron of de lamb' → trying fuzzy matching...
[Fuzzy] ❌ No match found (score=73)
🎤 Using input device: default
🎙 Please speak into the microphone...


Transcribing: 100%|██████████| 1/1 [00:00<00:00,  7.33it/s]


🗣 Command: 'turnn of the lamp'
[Adapt] ❌ No confident match for: 'turnn of the lamp' → trying fuzzy matching...
[Fuzzy] ✅ Matched 'turn off the lamp' (score=94)
🎤 Using input device: default
🎙 Please speak into the microphone...
🎤 Using input device: default
🎙 Please speak into the microphone...
🎤 Using input device: default
🎙 Please speak into the microphone...
🎤 Using input device: default
🎙 Please speak into the microphone...


Transcribing: 100%|██████████| 1/1 [00:00<00:00,  5.89it/s]


🗣 Command: 'huck'
[Adapt] ❌ No confident match for: 'huck' → trying fuzzy matching...
[Fuzzy] ❌ No match found (score=34)
🎤 Using input device: default
🎙 Please speak into the microphone...


In [1]:
stop()                               

NameError: name 'stop' is not defined